In [ ]:
import pandas as pd
import numpy as np
from tqdm.notebook import trange, tqdm
import psutil
import re
import os
import time
from datetime import datetime

In [ ]:
# Collect some CPU activity/thermal data
# Intuitively, we all know these are related.
# But can some of these predict the others? Reliably?

def get_cpu_speed():
    val = os.popen("vcgencmd measure_clock arm").read()
    return float(val[val.find("=")+1:]) / 10**9


def get_cpu_temp():
    val = os.popen("vcgencmd measure_temp").read()
    return float(re.search(r"temp=([\d\.]+)", val)[1])


def get_volts():
    val = os.popen("vcgencmd measure_volts").read()
    return float(re.search(r"volt=([\d\.]+)", val)[1])


fandir = "/sys/devices/platform/cooling_fan/hwmon/"
fan = ""
if os.path.isfile(fandir):
    fans = os.listdir(fandir)
    if fans:
        fan = os.path.join(fandir, fans[0], "fan1_input")

def get_fan_speed():
    speed = 0.0
    if fan:
        with open(fan) as file:
            speed = float(file.read())
    return speed


numsamples = 2500
samples = []
for i in trange(numsamples):
    samples.append([get_cpu_speed(), get_volts(), get_fan_speed(), psutil.cpu_percent(), psutil.getloadavg()[0], get_cpu_temp()])
    time.sleep(0.5)

In [ ]:
df = pd.DataFrame(samples)
df.columns = ['cpu_speed', 'volts', 'fan_speed', 'cpu_pct', 'load_avg', 'temp']
df.to_csv(f"readings-{datetime.today().strftime('%Y-%m-%d-%H.00')}.csv", index=False)